# SQL query from table names - Continued

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [2]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [13]:
context = [
    {'role': 'system', 'content': """
        CREATE TABLE employees (
            ID_Usr INT PRIMARY KEY,
            name VARCHAR(255)
        );

        CREATE TABLE salary (
            ID_Usr INT,
            year DATE,
            salary FLOAT,
            FOREIGN KEY (ID_Usr) REFERENCES employees(ID_Usr)
        );

        CREATE TABLE studies (
            ID INT PRIMARY KEY,
            ID_Usr INT,
            educational_level INT,
            Institution VARCHAR(255),
            Years DATE,
            Speciality VARCHAR(255),
            FOREIGN KEY (ID_Usr) REFERENCES employees(ID_Usr)
        );
    """},
    {'role': 'system', 'content': """
        -- Example queries for Few-Shot Learning:
        -- Get the institution whose graduates have the highest average salary.
        SELECT st.Institution, AVG(sa.salary) AS avg_salary
        FROM studies st
        JOIN employees e ON st.ID_Usr = e.ID_Usr
        JOIN salary sa ON e.ID_Usr = sa.ID_Usr
        GROUP BY st.Institution
        ORDER BY avg_salary DESC
        LIMIT 1;
    """}
]


In [14]:
#FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
 -- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.
WRITE IN YOUR CONTEXT QUERIES HERE
"""
})

In [15]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [16]:
#new
context_user = context.copy()
query = "Get the name of the employee with the highest salary"
print(return_CCRMSQL(query, context_user))


```sql
SELECT e.name
FROM employees e
JOIN salary sa ON e.ID_Usr = sa.ID_Usr
ORDER BY sa.salary DESC
LIMIT 1;
```


In [17]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("Get the name of the employee with the highest salary"
, old_context_user))

This is your SQL:
```sql
SELECT e.name
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
ORDER BY s.salary DESC
LIMIT 1;
```

This SQL query retrieves the name of the employee with the highest salary by joining the "employees" table with the "salary" table on the ID_usr column. It then orders the result by salary in descending order and limits the output to only the top result, which corresponds to the employee with the highest salary.


In [18]:
#new
print(return_CCRMSQL("Which institution has graduates with the highest average salary based on employee salary records?", context_user))

To find the institution with graduates having the highest average salary based on employee salary records, you can use the following SQL query:

```sql
SELECT st.Institution, AVG(sa.salary) AS avg_salary
FROM studies st
JOIN employees e ON st.ID_Usr = e.ID_Usr
JOIN salary sa ON e.ID_Usr = sa.ID_Usr
GROUP BY st.Institution
ORDER BY avg_salary DESC
LIMIT 1;
```

This query will calculate the average salary for graduates of each institution by joining the `studies`, `employees`, and `salary` tables. It will then group the results by institution, order them by average salary in descending order, and limit the output to the institution with the highest average salary.


In [21]:
#old
print(return_CCRMSQL("Which institution has graduates with the highest average salary?", old_context_user))


This is your SQL:
```sql
SELECT s.Institution, AVG(sa.salary) AS avg_salary
FROM studies s
JOIN employees e ON s.ID_usr = e.ID_usr
JOIN salary sa ON s.ID_usr = sa.ID_usr
GROUP BY s.Institution
ORDER BY avg_salary DESC
LIMIT 1;
```

This SQL query joins the tables `studies`, `employees`, and `salary` to calculate the average salary for graduates of each institution. It then selects the institution with the highest average salary by ordering the results in descending order and limiting the output to the top result.


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

In [22]:
store_context = [
    {'role': 'system', 'content': """
        CREATE TABLE products (
            product_id INT PRIMARY KEY,
            product_name VARCHAR(255),
            category VARCHAR(100),
            price FLOAT
        );

        CREATE TABLE sales (
            sale_id INT PRIMARY KEY,
            product_id INT,
            quantity INT,
            sale_date DATE,
            FOREIGN KEY (product_id) REFERENCES products(product_id)
        );
    """},
    {'role': 'system', 'content': """
        -- Example queries for Few-Shot Learning:
        -- Get the most popular fruit (highest quantity sold).
        SELECT p.product_name, SUM(s.quantity) AS total_sold
        FROM products p
        JOIN sales s ON p.product_id = s.product_id
        WHERE p.category = 'Fruit'
        GROUP BY p.product_name
        ORDER BY total_sold DESC
        LIMIT 1;
    """}
]


In [23]:
context_user = store_context.copy()
print(return_CCRMSQL("Which product generated the highest revenue?", context_user))


To determine which product generated the highest revenue, we need to calculate the total revenue generated by each product by multiplying the quantity sold by the price of the product. Here is the SQL query to find the product that generated the highest revenue:

```sql
SELECT p.product_name, SUM(s.quantity * p.price) AS total_revenue
FROM products p
JOIN sales s ON p.product_id = s.product_id
GROUP BY p.product_name
ORDER BY total_revenue DESC
LIMIT 1;
```

This query will calculate the total revenue generated by each product by multiplying the quantity sold by the price, then group the results by product name, and finally, return the product that generated the highest revenue.


In [24]:
context_user = store_context.copy()
print(return_CCRMSQL("Which fruit is the best-selling?", context_user))


To determine the best-selling fruit, we can run the following SQL query:

```sql
SELECT p.product_name, SUM(s.quantity) AS total_sold
FROM products p
JOIN sales s ON p.product_id = s.product_id
WHERE p.category = 'Fruit'
GROUP BY p.product_name
ORDER BY total_sold DESC
LIMIT 1;
```

This query will provide the fruit that has the highest quantity sold.


In [25]:
context_user = store_context.copy()
print(return_CCRMSQL("Which product is the least sold?", context_user))


To find the product that is the least sold, you can use the following query:

```sql
SELECT p.product_name, COALESCE(SUM(s.quantity), 0) AS total_sold
FROM products p
LEFT JOIN sales s ON p.product_id = s.product_id
GROUP BY p.product_name
ORDER BY total_sold
LIMIT 1;
```

This query will return the product that has the least quantity sold.


In [26]:
context_user = store_context.copy()
print(return_CCRMSQL("Which product was sold more than 5 times in the current month?", context_user))


To find the product that was sold more than 5 times in the current month, you can use the following SQL query:

```sql
SELECT p.product_name, SUM(s.quantity) AS total_sold
FROM products p
JOIN sales s ON p.product_id = s.product_id
WHERE MONTH(s.sale_date) = MONTH(CURRENT_DATE()) AND YEAR(s.sale_date) = YEAR(CURRENT_DATE())
GROUP BY p.product_name
HAVING total_sold > 5;
```

This query will retrieve the product name and the total quantity sold for products that were sold more than 5 times in the current month.


In [27]:
# creative 1

context_user = store_context.copy()
print(return_CCRMSQL("Which product had the highest monthly sales rate in the current year?", context_user))


To determine the product with the highest monthly sales rate in the current year, you can use the following SQL query:

```sql
SELECT p.product_name, MONTH(s.sale_date) AS month, SUM(s.quantity) AS total_sold
FROM products p
JOIN sales s ON p.product_id = s.product_id
WHERE YEAR(s.sale_date) = YEAR(CURRENT_DATE()) -- Filter for the current year
GROUP BY p.product_name, MONTH(s.sale_date)
ORDER BY total_sold DESC
LIMIT 1;
```

This query will calculate the total quantity sold for each product in each month of the current year, then identify the product with the highest monthly sales rate.


In [28]:
# creative 2

context_user = store_context.copy()
print(return_CCRMSQL("Which category generated the highest revenue this year?", context_user))


To determine which category generated the highest revenue this year, we need to calculate the revenue generated by each category by multiplying the quantity of each product sold by its price, and then summing up the total revenue for each category. Here is the SQL query to achieve this:

```sql
SELECT p.category, SUM(s.quantity * p.price) AS total_revenue
FROM products p
JOIN sales s ON p.product_id = s.product_id
WHERE YEAR(s.sale_date) = YEAR(CURDATE())
GROUP BY p.category
ORDER BY total_revenue DESC
LIMIT 1;
```

This query will calculate the total revenue generated by each category for sales that occurred this year, and then return the category that generated the highest revenue.


In [29]:
# creative 3

context_user = store_context.copy()
print(return_CCRMSQL("Which product saw the biggest drop in sales compared to last year?", context_user))


To determine the product that saw the biggest drop in sales compared to last year, we need to compare the total quantity sold for each product in the current year with the total quantity sold for each product in the previous year. However, the provided database schema does not include a column for the year of sale. To calculate the drop in sales compared to last year, we would need additional information such as the sale date or year.

If you can provide more details or modify the schema to include the necessary information, I can assist you further in analyzing the product that saw the biggest drop in sales compared to last year.
